In [149]:
from Common_Functions.CommonFunctions import *
from Patches_Functions.ExtractPatches import extract_patches
from sklearn.neighbors import NearestNeighbors
from skimage import transform
# from irls import irls

In [ ]:
def irls(X,z,r,Iirls,patch_size,subsampling_gap):
    Xp=extract_patches(X,patch_size,subsampling_gap)
    num_of_patches=(np.shape(Xp)[0]+1)*(np.shape(Xp)[1]+1)
    w=(np.ones((num_of_patches,1)))
    # z should be initialized with patch matching
    for i in range(0, Iirls):
        for x in range(0, np.shape(Xp)[0]):
            for y in range(0,np.shape(Xp)[1]):
                e=Xp[x,y,0,:,:,:]-z[x*np.shape(Xp)[0]+y]
                e2=np.sum(e**2)**0.5+0.00001
                w[x*np.shape(Xp)[0]+y]=(e2**(r-2))
                Xp[x,y,0,:,:,:]+=(z[x*np.shape(Xp)[0]+y]-Xp[x,y,0,:,:,:])*w[x*np.shape(Xp)[0]+y]

def style_transfer(content,style,r,L,Iirls,patch_sizes,subsampling_gaps, Ialg):
    # color transfer
    # gaussian pyramid
    # patch tree(optional)
    X= random_noise(content, mode="gaussian",mean=0,var=0.02)
    for l in range (L-1,-1,-1):
        for s in range(0,len(patch_sizes)):
            style_patches =  extract_patches(style,(patch_sizes[s], patch_sizes[s], 3),subsampling_gaps[s])
            flatten_style_patches = style_patches.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)
            nn_model = NearestNeighbors(n_neighbors=1).fit(flatten_style_patches)
            z=[]
            for k in range(0,Ialg):
                Xp=extract_patches(X,(patch_sizes[s], patch_sizes[s], 3),subsampling_gaps[s])
                flatten_Xp = Xp.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)
                for Xpatch in flatten_Xp:
                   Xpatch = [Xpatch]     
                   flatten_neighbour_patch = flatten_style_patches[nn_model.kneighbors(Xpatch)[1][0][0]]
                   # show_images([flatten_neighbour_patch.reshape(patch_sizes[s], patch_sizes[s], 3), Xpatch[0].reshape(patch_sizes[s], patch_sizes[s], 3)])
                   z.append(flatten_neighbour_patch.reshape(patch_sizes[s], patch_sizes[s], 3)) # we need to reshape the z to be n*n*3
                irls(X,z,r,Iirls,(patch_sizes[s],patch_sizes[s],3),subsampling_gaps[s]) 
                show_images([X])










style=io.imread("imgs/starry.jpg").astype(float)/255
X=io.imread("imgs/vat.jpg").astype(float)/255
# X = transform.resize(X, style.shape[:2], mode='constant', anti_aliasing=True)
X= random_noise(X, mode="gaussian",mean=0,var=0.02)
print(X.shape, style.shape)
# shape = 40
# Xp=extract_patches(X,(shape,shape,3),shape)
# style_patches =  extract_patches(style,(shape,shape,3),shape)
# z=[]
# style_patches = style_patches.reshape(-1, shape * shape * 3)
# nn_model = NearestNeighbors(n_neighbors=1).fit(style_patches)
# 
# print(X.shape)
# Xp = Xp.reshape(-1, shape * shape * 3)
# 
# print(Xp[0], style_patches[0])
# 
# print(4, (Xp.shape))
# for Xpatch in Xp:
# 
#     temp_arr = [Xpatch]
#     index = nn_model.kneighbors(temp_arr)[1][0][0] 
#     print(index, nn_model.kneighbors(temp_arr)[0][0][0])
#     paa = style_patches[index]
#     z.append(style_patches[index])
#     #now do un reshape to be 20*20*3
#     new_patch = paa.reshape(shape,shape,3)
#     show_images([new_patch, Xpatch.reshape(shape,shape,3)])

    # X = np.array(z) # should be replaced by the new xpatches generated from irls

# show_images([X])
style_transfer(X,style,1,1,10,[40],[40],1)